In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

In [2]:
# Read in data
mod_data = pd.read_csv('appointments_clean.csv')

In [3]:
# Split into X and y
X = mod_data.drop(
    [
        'PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 
        'Neighbourhood', 'No-show', 'NoShow'
    ], 
    axis = 1
)
y = mod_data.NoShow

# Test/train split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 314
)

In [4]:
# Grid search - gradient boosted
param_gb = {
    'loss': ['deviance'],
    'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    'min_samples_split': np.linspace(0.1, 0.5, 9),
    'min_samples_leaf': np.linspace(0.1, 0.5, 9),
    'max_depth': [3, 5, 8],
    'max_features': ['log2', 'sqrt'],
    'criterion': ['friedman_mse'],
    'subsample': np.linspace(0.5, 1.0, 5),
    'n_estimators': [10]
} 

grid_gb01 = RandomizedSearchCV(
    GradientBoostingClassifier(),
    param_gb,
    n_iter = 50,
    refit = True,
    verbose = 3
)

grid_gb02 = RandomizedSearchCV(
    GradientBoostingClassifier(),
    param_gb,
    n_iter = 50,
    scoring = 'brier_score_loss',
    refit = True,
    verbose = 3
)

# Grid search - neural net
param_mlp = {
    'solver': ['lbfgs', 'sgd'], 
    'max_iter': np.arange(10, 21) * 100, 
    'alpha': 10.0 ** -np.arange(1, 10), 
    'hidden_layer_sizes': np.arange(10, 16)
}

grid_mlp01 = RandomizedSearchCV(
    MLPClassifier(),
    param_mlp,
    n_iter = 50,
    refit = True,
    verbose = 3
)

grid_mlp02 = RandomizedSearchCV(
    MLPClassifier(),
    param_mlp,
    n_iter = 50,
    scoring = 'brier_score_loss',
    refit = True,
    verbose = 3
)

In [5]:
gb01 = grid_gb01.fit(X_train, y_train)
gb02 = grid_gb02.fit(X_train, y_train)
mlp01 = grid_mlp01.fit(X_train, y_train)
mlp02 = grid_mlp02.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] subsample=0.5, n_estimators=10, min_samples_split=0.4, min_samples_leaf=0.35, max_features=log2, max_depth=5, loss=deviance, learning_rate=0.1, criterion=friedman_mse 
[CV]  subsample=0.5, n_estimators=10, min_samples_split=0.4, min_samples_leaf=0.35, max_features=log2, max_depth=5, loss=deviance, learning_rate=0.1, criterion=friedman_mse, score=0.7991857309034509, total=   0.1s
[CV] subsample=0.5, n_estimators=10, min_samples_split=0.4, min_samples_leaf=0.35, max_features=log2, max_depth=5, loss=deviance, learning_rate=0.1, criterion=friedman_mse 
[CV]  subsample=0.5, n_estimators=10, min_samples_split=0.4, min_samples_leaf=0.35, max_features=log2, max_depth=5, loss=deviance, learning_rate=0.1, criterion=friedman_mse, score=0.7991779440846872, total=   0.1s
[CV] subsample=0.5, n_estimators=10, min_samples_split=0.4, min_samples_leaf=0.35, max_features=log2, max_depth=5, loss=deviance, learning_rate=0.1, criterion=fried

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV]  subsample=0.875, n_estimators=10, min_samples_split=0.15000000000000002, min_samples_leaf=0.45000000000000007, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.05, criterion=friedman_mse, score=0.7991857309034509, total=   0.1s
[CV] subsample=0.875, n_estimators=10, min_samples_split=0.15000000000000002, min_samples_leaf=0.45000000000000007, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.05, criterion=friedman_mse 
[CV]  subsample=0.875, n_estimators=10, min_samples_split=0.15000000000000002, min_samples_leaf=0.45000000000000007, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.05, criterion=friedman_mse, score=0.7991779440846872, total=   0.1s
[CV] subsample=0.875, n_estimators=10, min_samples_split=0.15000000000000002, min_samples_leaf=0.45000000000000007, max_features=log2, max_depth=3, loss=deviance, learning_rate=0.05, criterion=friedman_mse 
[CV]  subsample=0.875, n_estimators=10, min_samples_split=0.15000000000000002, min_samples

[CV]  subsample=0.75, n_estimators=10, min_samples_split=0.1, min_samples_leaf=0.15000000000000002, max_features=sqrt, max_depth=3, loss=deviance, learning_rate=0.075, criterion=friedman_mse, score=0.7991779440846872, total=   0.2s
[CV] subsample=0.75, n_estimators=10, min_samples_split=0.1, min_samples_leaf=0.15000000000000002, max_features=sqrt, max_depth=3, loss=deviance, learning_rate=0.075, criterion=friedman_mse 
[CV]  subsample=0.75, n_estimators=10, min_samples_split=0.1, min_samples_leaf=0.15000000000000002, max_features=sqrt, max_depth=3, loss=deviance, learning_rate=0.075, criterion=friedman_mse, score=0.7991779440846872, total=   0.2s
[CV] subsample=0.5, n_estimators=10, min_samples_split=0.15000000000000002, min_samples_leaf=0.2, max_features=log2, max_depth=8, loss=deviance, learning_rate=0.05, criterion=friedman_mse 
[CV]  subsample=0.5, n_estimators=10, min_samples_split=0.15000000000000002, min_samples_leaf=0.2, max_features=log2, max_depth=8, loss=deviance, learning_r

...

[CV]  solver=lbfgs, max_iter=1900, hidden_layer_sizes=13, alpha=0.0001, score=-0.05297948658753129, total=  20.2s
[CV] solver=lbfgs, max_iter=1600, hidden_layer_sizes=15, alpha=1e-06 .
[CV]  solver=lbfgs, max_iter=1600, hidden_layer_sizes=15, alpha=1e-06, score=-0.05550235014380075, total=  15.4s
[CV] solver=lbfgs, max_iter=1600, hidden_layer_sizes=15, alpha=1e-06 .
[CV]  solver=lbfgs, max_iter=1600, hidden_layer_sizes=15, alpha=1e-06, score=-0.05433242057282108, total=  12.1s
[CV] solver=lbfgs, max_iter=1600, hidden_layer_sizes=15, alpha=1e-06 .
[CV]  solver=lbfgs, max_iter=1600, hidden_layer_sizes=15, alpha=1e-06, score=-0.05304207242204104, total=  19.6s
[CV] solver=lbfgs, max_iter=1900, hidden_layer_sizes=12, alpha=1e-06 .
[CV]  solver=lbfgs, max_iter=1900, hidden_layer_sizes=12, alpha=1e-06, score=-0.055149558194005355, total=   7.6s
[CV] solver=lbfgs, max_iter=1900, hidden_layer_sizes=12, alpha=1e-06 .
[CV]  solver=lbfgs, max_iter=1900, hidden_layer_sizes=12, alpha=1e-06, score=-

[CV]  solver=lbfgs, max_iter=2000, hidden_layer_sizes=13, alpha=0.0001, score=-0.05321111453366941, total=  23.1s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=1e-08 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=1e-08, score=-0.05603827245973521, total=  11.8s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=1e-08 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=1e-08, score=-0.05730197893598838, total=   7.2s
[CV] solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=1e-08 ...
[CV]  solver=sgd, max_iter=1300, hidden_layer_sizes=14, alpha=1e-08, score=-0.05716445263467539, total=  10.7s
[CV] solver=sgd, max_iter=1000, hidden_layer_sizes=10, alpha=0.001 ...
[CV]  solver=sgd, max_iter=1000, hidden_layer_sizes=10, alpha=0.001, score=-0.055875789379471005, total=   5.4s
[CV] solver=sgd, max_iter=1000, hidden_layer_sizes=10, alpha=0.001 ...
[CV]  solver=sgd, max_iter=1000, hidden_layer_sizes=10, alpha=0.001, score=-0.05502733

[CV]  solver=sgd, max_iter=1400, hidden_layer_sizes=13, alpha=0.01, score=-0.05493843084628222, total=   7.5s
[CV] solver=sgd, max_iter=1800, hidden_layer_sizes=15, alpha=1e-05 ...
[CV]  solver=sgd, max_iter=1800, hidden_layer_sizes=15, alpha=1e-05, score=-0.056210780970635026, total=   8.4s
[CV] solver=sgd, max_iter=1800, hidden_layer_sizes=15, alpha=1e-05 ...
[CV]  solver=sgd, max_iter=1800, hidden_layer_sizes=15, alpha=1e-05, score=-0.05500413469434989, total=   8.5s
[CV] solver=sgd, max_iter=1800, hidden_layer_sizes=15, alpha=1e-05 ...
[CV]  solver=sgd, max_iter=1800, hidden_layer_sizes=15, alpha=1e-05, score=-0.053704439968650625, total=  10.7s
[CV] solver=sgd, max_iter=2000, hidden_layer_sizes=12, alpha=0.1 .....
[CV]  solver=sgd, max_iter=2000, hidden_layer_sizes=12, alpha=0.1, score=-0.05597694517047195, total=  11.5s
[CV] solver=sgd, max_iter=2000, hidden_layer_sizes=12, alpha=0.1 .....
[CV]  solver=sgd, max_iter=2000, hidden_layer_sizes=12, alpha=0.1, score=-0.055545191978325

[CV]  solver=sgd, max_iter=1400, hidden_layer_sizes=14, alpha=0.001, score=-0.05408358497150171, total=  11.1s
[CV] solver=sgd, max_iter=1400, hidden_layer_sizes=14, alpha=0.0001 ..
[CV]  solver=sgd, max_iter=1400, hidden_layer_sizes=14, alpha=0.0001, score=-0.05566205552133878, total=  10.9s
[CV] solver=sgd, max_iter=1400, hidden_layer_sizes=14, alpha=0.0001 ..
[CV]  solver=sgd, max_iter=1400, hidden_layer_sizes=14, alpha=0.0001, score=-0.06032944671179578, total=   5.2s
[CV] solver=sgd, max_iter=1400, hidden_layer_sizes=14, alpha=0.0001 ..
[CV]  solver=sgd, max_iter=1400, hidden_layer_sizes=14, alpha=0.0001, score=-0.054414688059185465, total=   6.5s
[CV] solver=lbfgs, max_iter=1700, hidden_layer_sizes=15, alpha=0.001 .
[CV]  solver=lbfgs, max_iter=1700, hidden_layer_sizes=15, alpha=0.001, score=-0.055486807275129627, total=  16.8s
[CV] solver=lbfgs, max_iter=1700, hidden_layer_sizes=15, alpha=0.001 .
[CV]  solver=lbfgs, max_iter=1700, hidden_layer_sizes=15, alpha=0.001, score=-0.054

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 27.7min finished


In [6]:
def prob_summary(clf, data, actual):
    
    # Generate predicted probabilities
    probs = clf.predict_proba(data)
    probs = pd.Series([i[1] for i in probs])
    
    # Add probabilities to actuals in dataframe
    probs_df = pd.concat([probs, actual.reset_index()], axis = 1).reset_index()
    
    # Clean dataframe
    probs_df.columns = ['level_0', 'Prob', 'index', 'Actual']
    probs_df         = probs_df.drop(['level_0', 'index'], axis = 1)
    
    # Round probabilities to nearest tenth
    probs_df['RoundedProb'] = (probs_df['Prob']).round(1)
    
    # Summarize
    summary = probs_df.groupby(
        'RoundedProb'
    ).agg(
        {
            'Actual':['count', 'sum']
        }
    )
    
    # Fix column names
    summary.columns = ["_".join(i) for i in summary.columns.ravel()]
    
    # Calculate predicted percentages
    summary['Actual_percentage'] = (summary['Actual_sum'] / summary['Actual_count']).round(2)
    return(summary)

In [7]:
print('Gradient Boosting')
print(prob_summary(gb01, X_test, y_test), '\n')

print('Gradient Boosting - Brier Score Loss')
print(prob_summary(gb01, X_test, y_test), '\n')

print('Neural Network')
print(prob_summary(mlp01, X_test, y_test), '\n')

print('Neural Network - Brier Score Loss')
print(prob_summary(mlp01, X_test, y_test), '\n')

Gradient Boosting
             Actual_count  Actual_sum  Actual_percentage
RoundedProb                                             
0.2                 33159        6782                0.2 

Gradient Boosting - Brier Score Loss
             Actual_count  Actual_sum  Actual_percentage
RoundedProb                                             
0.2                 33159        6782                0.2 

Neural Network
             Actual_count  Actual_sum  Actual_percentage
RoundedProb                                             
0.0                 23091           0               0.00
0.1                    41          13               0.32
0.2                   791         159               0.20
0.3                   610         172               0.28
0.4                   266         106               0.40
0.5                    87          44               0.51
0.6                   946         621               0.66
0.7                  2873        1992               0.69
0.8           

In [8]:
with open("Schedule_gb01.pkl", 'wb') as file:  
    pickle.dump(gb01, file)
    
with open("Schedule_gb02.pkl", 'wb') as file:  
    pickle.dump(gb02, file)
    
with open("Schedule_mlp01.pkl", 'wb') as file:  
    pickle.dump(mlp01, file)
    
with open("Schedule_mlp02.pkl", 'wb') as file:  
    pickle.dump(mlp02, file)